# Cleaning of shark attacks data set

In [1]:
#Imports
import pandas as pd
import numpy as np
import re

# Functions
# Evaluate the NA's
def evaluar_NA(data):
    # Pandas series denoting features and the sum of their null values
    null_sum = data.isna().sum()
    # Total
    total = null_sum.sort_values(ascending=False)
    # Percentage
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False) 
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])   
    return df_NA

# data
data = pd.read_csv('../data/attacks.csv', encoding='latin-1')

Quick preview of the data

In [2]:
print(data.shape)
print(data.columns)


(25723, 24)
Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


Our first task is to subset the initial dataset to keep only the variables that we are going to use:
* Type
* Country
* Activity
* Sex
* Age
* Species

In [3]:
# Subset the dataframe with the needed columns
df_subset = data[["Case Number", "Type", "Country", "Area", "Activity", "Sex ", "Age", "Species "]]

# Rename the Sex and Species columns to remove the last space and avoid problems in the future...
df_subset = df_subset.rename(columns={"Sex ": "Sex", "Species ": "Species"})

df_subset.head()

,Case Number,Type,Country,Area,Activity,Sex,Age,Species
0,2018.06.25,Boating,USA,California,Paddling,F,57,White shark
1,2018.06.18,Unprovoked,USA,Georgia,Standing,F,11,NaN
2,2018.06.09,Invalid,USA,Hawaii,Surfing,M,48,NaN
3,2018.06.08,Unprovoked,AUSTRALIA,New South Wales,Surfing,M,NaN,2 m shark
4,2018.06.04,Provoked,MEXICO,Colima,Free diving,M,NaN,"Tiger shark, 3m"


### The most conflictive human

In [4]:
# Select only the needed columns
df_humans = df_subset[["Type", "Country", "Activity", "Sex", "Age",]]
# Select only the provoked incidents
df_humans = df_humans[df_humans["Type"] == "Provoked"]
#df_humans

First at all, we are goint the evaluate the amount of NA's in the dataset

In [5]:
# Evaluate the percentage of 
evaluar_NA(df_humans)

,Number of NA,Percent NA
Age,294,51.22
Sex,57,9.93
Activity,35,6.10
Country,3,0.52
Type,0,0.00


Due to the high percentage of NA's in the **Age** column, we've decided to exclude it from the dataset.

In [6]:
# Drop the "Age" column
df_humans_cl = df_humans.drop(["Age"], axis=1)

# Drop NA's
df_humans_cl = df_humans_cl.dropna(axis=0)
df_humans_cl.shape

(487, 4)

Now, throgh regex expressions, we are going to extract the activy. With this regex, we want to group the activities into a few categories, in order to better group the data.

In [7]:
df_humans_cl["regActivity"] = df_humans_cl["Activity"].str.findall(r"((?:[A-Za-z-]*\s){0,1}(?:[A-Za-z]*ing))")
#print(df_humans_cl["regActivity"].value_counts())
df_humans_cl.iloc[:10, ]

,Type,Country,Activity,Sex,regActivity
4,Provoked,MEXICO,Free diving,M,[Free diving]
10,Provoked,AUSTRALIA,Feeding sharks,M,[Feeding]
14,Provoked,AUSTRALIA,Feeding sharks,F,[Feeding]
41,Provoked,AUSTRALIA,Kayak fishing for sharks,M,[Kayak fishing]
55,Provoked,MALAYSIA,Fishing / Wading,M,"[Fishing, Wading]"
104,Provoked,USA,Fishing,M,[Fishing]
122,Provoked,MEXICO,Fishing,M,[Fishing]
139,Provoked,USA,Touching a shark,F,[Touching]
143,Provoked,AUSTRALIA,Attempting to lasso a shark,M,[Attempting]
177,Provoked,BRAZIL,Grabbing shark for a selfie,F,[Grabbing]


In [8]:
# Drop possible empty lists
df_humans_cl = df_humans_cl[df_humans_cl["regActivity"].str.len() != 0]

# Select only the first element on the "regActivity" list
df_humans_cl["regActivity"] = df_humans_cl["regActivity"].apply(lambda x: x[0] )

# Convert the regActivity list to string
df_humans_cl["regActivity"] = df_humans_cl["regActivity"].apply(''.join)
df_humans_cl.iloc[:10, ]

,Type,Country,Activity,Sex,regActivity
4,Provoked,MEXICO,Free diving,M,Free diving
10,Provoked,AUSTRALIA,Feeding sharks,M,Feeding
14,Provoked,AUSTRALIA,Feeding sharks,F,Feeding
41,Provoked,AUSTRALIA,Kayak fishing for sharks,M,Kayak fishing
55,Provoked,MALAYSIA,Fishing / Wading,M,Fishing
104,Provoked,USA,Fishing,M,Fishing
122,Provoked,MEXICO,Fishing,M,Fishing
139,Provoked,USA,Touching a shark,F,Touching
143,Provoked,AUSTRALIA,Attempting to lasso a shark,M,Attempting
177,Provoked,BRAZIL,Grabbing shark for a selfie,F,Grabbing


Create the final dataset

In [9]:
# Select the final columns and rename them
humans = df_humans_cl[["Country", "Sex", "regActivity"]].rename(columns = {"regActivity" : "Activity"}).reset_index(drop=True)
#humans
# Export the data
humans.to_csv("output/humans.csv")

### The most trouble maker shark by species and region

In [10]:
df_sharks = df_subset[["Country", "Area", "Species"]]

In [11]:
# Check NAs
evaluar_NA(df_sharks)

,Number of NA,Percent NA
Species,22259,86.53
Area,19876,77.27
Country,19471,75.69


In [12]:
# Drop "Species" rows with null values
df_sharks_cl = df_sharks.dropna(axis=0)
df_sharks_cl.shape


(3300, 3)

Extracting the shark species

In [13]:
#prueba
# Extract from 1 up to 2 words before the word 'shark'
df_sharks_cl['spp'] = df_sharks_cl["Species"].str.findall(r"((?:[A-Za-z-]*\s){1,2}(?:[Ss]hark|[Cc]atfish|[Pp]ointer))")
df_sharks_cl.iloc[0:10, [0,1,2]]

/home/lee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Area,Species
0,USA,California,White shark
3,AUSTRALIA,New South Wales,2 m shark
4,MEXICO,Colima,"Tiger shark, 3m"
6,BRAZIL,Pernambuco,Tiger shark
7,USA,Florida,"Lemon shark, 3'"
8,USA,Florida,"Bull shark, 6'"
10,AUSTRALIA,Queensland,Grey reef shark
13,ENGLAND,Cornwall,Invalid incident
14,AUSTRALIA,Westerm Australia,"Tawny nurse shark, 2m"
16,AUSTRALIA,New South Wales,Shark involvement not confirmed


In [14]:
# Extract only the values wich are not empty
df_sharks_cl = df_sharks_cl[df_sharks_cl["spp"].str.len() != 0]
df_sharks_cl.iloc[1, [2]]

Species    2 m shark
Name: 3, dtype: object

In [15]:
# Convert to strings
df_sharks_cl["spp_str"] = df_sharks_cl["spp"].apply(''.join)

# Delete rows with only 'shark', 'm shark', 'No shark', 'A small shark'
df_sharks_cl2 = df_sharks_cl[df_sharks_cl["spp_str"].isin([' m shark', 'm shark', ' shark', 'No shark','Not a shark', 'not a shark','A small shark', 'a small shark'])== False]

# Remove empty spaces
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.strip()
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.rstrip()

# Remove garbage strings at the beginning and the e
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.lstrip("A")
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.lstrip("or")
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.lstrip("juvenile")
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.lstrip("Juvenile")
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.lstrip("female")
# Remove empty spaces
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.strip()
# Capitalize the string
df_sharks_cl2["spp_str"] = df_sharks_cl2["spp_str"].str.capitalize()
df_sharks_cl2.iloc[:50, 3]

/home/lee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/lee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/lee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

0                  [White shark]
4                  [Tiger shark]
6                  [Tiger shark]
7                  [Lemon shark]
8                   [Bull shark]
10             [Grey reef shark]
14           [Tawny nurse shark]
18                 [Tiger shark]
22                 [White shark]
23                 [Tiger shark]
24                 [White shark]
26         [Juvenile bull shark]
30                 [Tiger shark]
34                 [White shark]
35                 [Tiger shark]
36             [Wobbegong shark]
39              [Blacktip shark]
40        [Juvenile white shark]
42                  [Bull shark]
45             [Galapagos shark]
47                  [Bull shark]
49             [Grey reef shark]
50                 [White shark]
51                 [small shark]
52             [Wobbegong shark]
53                 [Tiger shark]
55        [Juvenile nurse shark]
57                 [Tiger shark]
58                 [Tiger shark]
60                 [White shark]
64        

In [16]:
# Final 
sharks = df_sharks_cl2[["Country","Area","spp_str"]]

# Rename and reset index
sharks = sharks.rename(columns = {"spp_str" : "Species"}).reset_index(drop=True)

# Export to csv
sharks.to_csv("output/sharks.csv")